In [ ]:
# Required for importing modules from parent directory
import os
import sys

current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(current_dir)
print(parent_dir)
sys.path.append(parent_dir)

In [ ]:
from pathlib import Path

import soccer_twos

from src.agents.ddpg_agent import DDPGAgents
from src.agents.ddqn_agent import DDQNAgents
from src.agents.maddpg_agent import MADDPGAgents
from src.config import N_GAMES
from src.utils import shape_rewards

CHECKPOINT_DIR = "notebooks/data/kaggle_single_out_1000/checkpoints"

In [ ]:
def evaluate_agent(agent, n_games=N_GAMES):
    env = soccer_twos.make(
        render=True,
        time_scale=1,
        quality_level=5,
    )

    for i in range(n_games):
        obs = env.reset()
        done = False
        score = 0
        while not done:

            actions = {}
            for i in range(4):
                action = agent.act({0: obs[i]})
                actions[i] = action.pop(0)
                print(action)
            print(actions)
            next_obs, reward, done, info = env.step(actions)
            done = done["__all__"]
            obs = next_obs
            score = reward[0] + shape_rewards(info, 0)
            # print(shape_rewards(info, 0))
            env.render()  # Render the environment to visualize the game

In [ ]:
import glob

# Load the best agent (example: checkpoint at episode 90)
best_agent = DDQNAgents(1, 336, 3)

agent_name = best_agent.__class__.__name__.lower().replace("agents", "")
pattern = f"best_model_{agent_name}"

matching_files = glob.glob(str(parent_dir / Path(CHECKPOINT_DIR) / f"{pattern}*"))
filename = matching_files[0] if matching_files else None

print(matching_files)
best_agent.load(parent_dir / Path(CHECKPOINT_DIR) / str(matching_files[0]))

# Evaluate the trained agent
evaluate_agent(best_agent, n_games=5)